In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=5555)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13644 ( 0.0%) Users that have less than 1 test interactions


In [13]:
import os

ofp = "../all_models/URM_all/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'UserKNN' : {'topK': 448, 'similarity': 'cosine', 'shrink': 756, 'normalize': True, 'feature_weighting': 'TF-IDF', 'URM_bias': True},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"}
                         }


In [14]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [15]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

In [16]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender

In [20]:
recommender1 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_weighted)
recommender2 = IALSRecommender(URM_all)
recommender3 = RP3betaRecommender(URM_all)

model_init(recommender1, 'SLIM_weig', models_to_combine_best['SLIMweig'])
model_init(recommender2, 'IALS', models_to_combine_best['IALS'])
model_init(recommender3, 'RP3beta', models_to_combine_best['RP3beta'])

../all_models/URM_all/SLIM_weig.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../all_models/URM_all/SLIM_weig'
SLIMElasticNetRecommender: Loading complete
../all_models/URM_all/IALS.zip
Model found!
IALSRecommender: Loading model from file '../all_models/URM_all/IALS'
IALSRecommender: Loading complete
../all_models/URM_all/RP3beta.zip
Model found!
RP3betaRecommender: Loading model from file '../all_models/URM_all/RP3beta'
RP3betaRecommender: Loading complete


In [34]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [35]:
def rank_models_evaluator_based(evaluators, recommenders):
    j = 0
    for e in evaluators:
        for r in recommenders:
            r_d, _ = e.evaluateRecommender(r)
            print("GROUP " + str(j), r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])
        j += 1

In [21]:
best_hyp = {'norm': 2,'alpha': 0.6021189514861884, 'beta': 0.055245873164717185, 'gamma': 0.24380474750485243}

In [22]:
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender

recommender_final = ThreeDifferentModelRecommender(URM_all.tocsr(), recommender1, recommender2, recommender3)
recommender_final.fit(**best_hyp)

CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.6021189514861884
IALSRecommender with weight beta: 0.055245873164717185
RP3betaRecommender with weight gamma: 0.24380474750485243
Norm type: 2


In [23]:
import pandas as pd
K=10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
                                sep=",",
                                dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
    subm_res["user_id"].append(user_id)
    res = recommender_final.recommend(user_id, K)
    print("USER", user_id, "RECOMMENDED!")
    res = ' '.join(map(str, res))
    print(res)
    subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

USER 0 RECOMMENDED!
3899 5433 16365 16359 8465 6061 9130 2243 640 10586
USER 1 RECOMMENDED!
3473 940 14400 16012 640 16365 10118 2118 9373 14358
USER 2 RECOMMENDED!
6177 1551 17330 17022 16365 15119 14229 5925 2665 4607
USER 3 RECOMMENDED!
8194 15551 16359 15082 960 8371 15507 16444 3910 5885
USER 4 RECOMMENDED!
8194 6177 2152 3473 5299 9871 6011 10954 10586 16494
USER 5 RECOMMENDED!
9621 1846 16640 14396 7867 11227 13098 13585 5312 99
USER 6 RECOMMENDED!
6214 985 16846 14358 14181 8207 7606 9621 16640 14868
USER 7 RECOMMENDED!
13963 15119 14031 4292 1885 3433 7751 17364 640 3768
USER 8 RECOMMENDED!
8635 3619 5605 2665 1259 10865 3768 5299 4853 8502
USER 9 RECOMMENDED!
5605 8207 1614 14181 13318 3473 1551 15117 14078 8936
USER 10 RECOMMENDED!
3899 6214 14358 14400 6975 16365 13229 5273 4607 13410
USER 11 RECOMMENDED!
14181 3473 10865 11359 249 12390 13134 6431 10370 5577
USER 12 RECOMMENDED!
3899 6724 3200 1137 1792 15778 14358 13134 5925 1259
USER 13 RECOMMENDED!
8635 6214 15119 14358

USER 134 RECOMMENDED!
16846 2243 15583 11844 2665 12943 17330 6011 12708 13134
USER 135 RECOMMENDED!
7264 3200 3910 1259 9871 7486 6011 14536 6975 5273
USER 136 RECOMMENDED!
4088 840 11702 7407 3200 2680 12028 4093 3619 14868
USER 137 RECOMMENDED!
14358 7143 3899 6011 15844 960 5981 2886 4582 14396
USER 138 RECOMMENDED!
8096 8635 3899 7257 9686 4301 13714 1838 873 13638
USER 139 RECOMMENDED!
3899 7407 8635 1941 2756 5925 1551 13963 15778 6214
USER 140 RECOMMENDED!
6214 6652 13914 8207 16365 15778 1137 13410 6975 15583
USER 141 RECOMMENDED!
640 6214 8207 6589 6011 16494 12901 13410 5299 4582
USER 142 RECOMMENDED!
15119 17584 640 4582 10562 3768 14917 4088 11359 12457
USER 143 RECOMMENDED!
1941 3899 14181 14485 7309 15778 10370 8326 10295 16364
USER 144 RECOMMENDED!
16070 1653 8635 13725 15778 1259 6782 9871 6214 1614
USER 145 RECOMMENDED!
13134 10865 13465 15674 12281 1137 1941 1044 1533 11553
USER 146 RECOMMENDED!
17788 8194 5885 5981 15082 15551 10871 16359 6061 7719
USER 147 RECOMMEN

USER 265 RECOMMENDED!
17622 2669 17788 8194 15289 11227 17297 16549 11504 15551
USER 266 RECOMMENDED!
3899 14037 3473 16624 6214 6975 16365 5640 8635 5324
USER 267 RECOMMENDED!
16365 6652 13963 13032 15868 17364 13134 6011 13914 249
USER 268 RECOMMENDED!
6177 14181 1259 16791 7295 7716 14358 16070 1137 17294
USER 269 RECOMMENDED!
6214 8635 14868 13585 17291 10865 15778 13914 13295 8473
USER 270 RECOMMENDED!
12707 3473 8635 4078 13134 16791 16846 10985 5273 12943
USER 271 RECOMMENDED!
3200 6306 7716 17961 249 3942 1846 17748 3829 8635
USER 272 RECOMMENDED!
6214 17330 14358 5273 249 14181 3889 15119 16846 6011
USER 273 RECOMMENDED!
3899 6177 1137 2152 12492 249 17748 14485 10370 11491
USER 274 RECOMMENDED!
11506 10427 17761 3473 2711 11259 5082 3768 435 8635
USER 275 RECOMMENDED!
6157 6214 13295 17330 11491 14181 2251 2796 15119 10865
USER 276 RECOMMENDED!
1614 8635 10562 3942 11339 13410 3619 13134 16012 10865
USER 277 RECOMMENDED!
442 7257 14520 5237 3473 16762 5934 4435 2711 13532
USE

USER 393 RECOMMENDED!
16364 8635 14400 5925 15778 5424 15119 13134 17574 7295
USER 394 RECOMMENDED!
1941 14868 16070 10865 1551 3899 16012 241 5424 12457
USER 395 RECOMMENDED!
2387 12752 10427 442 3899 12380 435 3473 11506 11259
USER 396 RECOMMENDED!
5273 3899 16365 3473 6975 8207 3942 3619 8326 14400
USER 397 RECOMMENDED!
8635 17330 12457 1941 17584 5403 1208 15778 1259 5925
USER 398 RECOMMENDED!
249 8502 5925 17526 10865 2587 3325 5423 17135 7295
USER 399 RECOMMENDED!
10008 7645 5300 7264 12983 8635 2871 4220 4562 9270
USER 400 RECOMMENDED!
3899 12186 12943 640 13134 940 5273 12707 7295 4582
USER 401 RECOMMENDED!
8194 17788 17330 5981 5433 7143 5273 15119 16365 13801
USER 402 RECOMMENDED!
1941 241 3473 8635 7407 16012 17526 13638 8942 4582
USER 403 RECOMMENDED!
15119 10865 4582 6975 1792 13134 5640 6011 17666 5925
USER 404 RECOMMENDED!
8635 10562 13318 14868 13032 13178 6873 5403 15119 3768
USER 405 RECOMMENDED!
3473 6214 6652 13963 17330 8635 17140 12390 2387 5273
USER 406 RECOMMEND

USER 522 RECOMMENDED!
6177 7309 3869 5605 3200 6724 12186 13318 8502 13584
USER 523 RECOMMENDED!
10562 1137 5273 3899 12457 14400 17135 13638 3829 2748
USER 524 RECOMMENDED!
16486 1551 17135 1137 3942 12457 13914 5424 4088 2756
USER 525 RECOMMENDED!
7264 15119 4435 3899 17391 2243 14358 14663 13921 11259
USER 526 RECOMMENDED!
14396 7606 16846 1551 5951 17330 101 14311 16486 9621
USER 527 RECOMMENDED!
940 2886 16365 4582 2152 5742 16070 16846 6975 7870
USER 528 RECOMMENDED!
7407 1941 10370 16919 13229 3899 16486 8635 14358 16846
USER 529 RECOMMENDED!
14125 12707 13725 13229 17936 3768 2152 6989 2126 10985
USER 530 RECOMMENDED!
6157 99 1846 3198 8580 17715 11430 3691 17411 9898
USER 531 RECOMMENDED!
14358 13914 10865 249 6177 3899 17936 15969 10295 1386
USER 532 RECOMMENDED!
1551 2886 17936 14358 1259 2152 16846 14400 8651 7295
USER 533 RECOMMENDED!
9621 6151 17622 4075 2669 15119 6392 13098 14917 17416
USER 534 RECOMMENDED!
15778 14358 5925 14266 14181 14400 4204 2846 3942 2665
USER 535

...

USER 13094 RECOMMENDED!
13725 8502 5925 6214 241 13465 3569 6905 1259 6782
USER 13095 RECOMMENDED!
16624 6214 16846 5324 985 13963 16365 6975 6652 3473
USER 13096 RECOMMENDED!
3473 15119 13134 14358 17388 9871 12186 13229 8207 241
USER 13097 RECOMMENDED!
9131 5885 4292 3899 16486 13229 5273 16365 1792 2545
USER 13098 RECOMMENDED!
12752 13229 6177 6157 1846 6975 9131 4292 2152 4582
USER 13099 RECOMMENDED!
4582 16846 15119 12172 3473 5640 3899 10295 16365 2746
USER 13100 RECOMMENDED!
16791 640 15778 5974 4582 14266 4853 3942 16365 8635
USER 13101 RECOMMENDED!
6214 8635 16624 14396 14358 10865 2665 12901 5951 7867
USER 13102 RECOMMENDED!
14358 4853 6177 17462 14078 17416 14266 17666 13914 5925
USER 13103 RECOMMENDED!
3899 17788 5273 16846 8011 4204 3889 17294 16486 13134
USER 13104 RECOMMENDED!
6214 16365 2665 15181 1792 15778 4582 17135 9131 6011
USER 13105 RECOMMENDED!
7716 13178 17330 13318 7973 9322 6670 14868 13914 15469
USER 13106 RECOMMENDED!
8635 15119 10865 4199 2482 3668 6495 98

USER 13228 RECOMMENDED!
202 1997 11506 7089 3080 5141 8917 13068 11576 2532
USER 13229 RECOMMENDED!
13296 12764 15119 14396 3768 4199 8580 11058 17022 17413
USER 13230 RECOMMENDED!
16846 16365 3473 6214 3899 640 14917 1792 17294 4292
USER 13231 RECOMMENDED!
2665 1792 13229 13134 5273 14181 7744 5640 17584 4078
USER 13232 RECOMMENDED!
6417 16962 14396 16846 12186 3899 53 14358 3668 13585
USER 13233 RECOMMENDED!
7744 13296 13585 6351 53 2552 8580 8194 17294 14396
USER 13234 RECOMMENDED!
3899 7645 15750 17788 9871 10865 14181 12707 5951 15979
USER 13235 RECOMMENDED!
14181 13691 1208 6905 13584 13914 7407 940 12828 10295
USER 13236 RECOMMENDED!
14358 2243 6011 5403 13410 14868 15778 1619 3829 5640
USER 13237 RECOMMENDED!
14181 15119 1137 5424 12707 6782 7744 985 1792 8502
USER 13238 RECOMMENDED!
6177 5403 15778 13134 7716 12186 1551 1259 6905 249
USER 13239 RECOMMENDED!
16846 12752 4607 2243 3768 2886 8868 17666 4661 12015
USER 13240 RECOMMENDED!
6873 14358 6177 4582 4853 6214 5925 1137 12

USER 13368 RECOMMENDED!
15778 1941 8635 15119 5925 13032 17330 12901 4582 6905
USER 13369 RECOMMENDED!
3473 8635 17330 9871 15778 2665 3691 2076 12186 4199
USER 13370 RECOMMENDED!
2152 10954 3768 6652 2665 6011 10562 3889 13914 3130
USER 13371 RECOMMENDED!
16846 16365 15119 15181 5324 6589 2886 3768 1792 8207
USER 13372 RECOMMENDED!
13914 1259 8635 14868 8651 1551 12457 3829 10562 1941
USER 13373 RECOMMENDED!
7295 3899 3569 14868 15750 512 3829 17022 9441 12378
USER 13374 RECOMMENDED!
985 6214 16846 17294 14358 3473 2665 8207 12186 3768
USER 13375 RECOMMENDED!
12457 12028 11051 6670 1941 13318 15082 10370 10758 17788
USER 13376 RECOMMENDED!
7867 15289 10865 3768 1551 1259 5299 12186 3829 4582
USER 13377 RECOMMENDED!
16365 640 12901 2846 15778 4582 1619 6011 17294 1883
USER 13378 RECOMMENDED!
16365 3768 15119 3473 12901 5324 13963 6976 8635 4582
USER 13379 RECOMMENDED!
6011 4292 2665 6975 14358 3473 8635 10954 13134 16364
USER 13380 RECOMMENDED!
15119 16946 7744 12901 1137 2665 4292 458

USER 13497 RECOMMENDED!
15119 5141 3473 3899 14358 17751 2687 11303 6540 15778
USER 13498 RECOMMENDED!
14917 3473 1614 13387 17748 17584 2387 9776 17330 12764
USER 13499 RECOMMENDED!
3899 8635 5605 17356 9871 7309 17936 9744 1614 3942
USER 13500 RECOMMENDED!
3473 16549 1551 13387 17297 3768 4582 803 8502 12457
USER 13501 RECOMMENDED!
3619 8942 1941 8651 15778 16364 15064 249 12774 1137
USER 13502 RECOMMENDED!
5951 99 6976 9898 3473 16846 4199 13585 8207 11788
USER 13503 RECOMMENDED!
3473 8898 15922 8573 17751 1137 15773 16313 6730 10132
USER 13504 RECOMMENDED!
6177 15119 17330 8635 11339 6975 12186 11359 2886 5299
USER 13505 RECOMMENDED!
6177 8207 4582 3473 3768 6011 17330 7983 16364 2243
USER 13506 RECOMMENDED!
17830 3899 16365 6011 15119 12943 6589 12707 5925 2886
USER 13507 RECOMMENDED!
12492 6177 9871 17022 14868 5403 16791 7412 16846 8448
USER 13508 RECOMMENDED!
14181 6652 3619 7486 13134 16364 14358 3579 2126 1899
USER 13509 RECOMMENDED!
7309 2179 15778 16486 16365 15082 13229 24

USER 13623 RECOMMENDED!
3899 14358 2665 3473 14181 13229 14868 14781 5640 15800
USER 13624 RECOMMENDED!
13229 6177 16365 5951 8635 10865 1846 17830 1614 985
USER 13625 RECOMMENDED!
6177 3899 985 13229 8635 12901 8011 9131 5299 17135
USER 13626 RECOMMENDED!
3473 14181 14213 6214 17330 1137 10843 7597 249 9776
USER 13627 RECOMMENDED!
5273 2665 16365 6214 13134 6905 12186 8194 8635 14400
USER 13628 RECOMMENDED!
17330 5273 13318 1941 6975 3200 13387 16012 14181 14400
USER 13629 RECOMMENDED!
8635 10865 5403 1619 15119 640 17294 6589 11491 525
USER 13630 RECOMMENDED!
17670 13032 17284 8986 12186 12901 16365 1792 6214 14868
USER 13631 RECOMMENDED!
14396 7867 6976 7606 16962 1551 8194 8473 8011 14868
USER 13632 RECOMMENDED!
15436 2084 1137 3889 11491 17526 17961 3829 12390 15469
USER 13633 RECOMMENDED!
12707 2772 10173 6670 14400 16791 14213 6214 15119 6652
USER 13634 RECOMMENDED!
7744 6214 13134 1941 99 17584 6417 13296 16640 9621
USER 13635 RECOMMENDED!
16365 3473 8011 12901 5885 15119 15082